In [8]:
import requests
import itertools
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Point

In [9]:
BASE_URL = 'http://127.0.0.1:5000/route/v1/driving/{};{}?steps=false&geometries=geojson&overview=simplified'

puntos = pd.read_csv('data/coords.csv')
puntos_str = [str(puntos.lon[i]) + ',' + str(puntos.lat[i]) for i in range(puntos.shape[0])]

puntos_str[0:3]

['-56.174030195714,-34.8634731144801',
 '-56.8155472789126,-30.4070624167322',
 '-56.0461527421891,-34.7048062340561']

In [10]:
product = list(itertools.product(puntos_str, puntos_str))
len(product)

361

In [11]:
urls = [BASE_URL.format(i[0], i[1]) for i in product]
urls[0:3]

['http://127.0.0.1:5000/route/v1/driving/-56.174030195714,-34.8634731144801;-56.174030195714,-34.8634731144801?steps=false&geometries=geojson&overview=simplified',
 'http://127.0.0.1:5000/route/v1/driving/-56.174030195714,-34.8634731144801;-56.8155472789126,-30.4070624167322?steps=false&geometries=geojson&overview=simplified',
 'http://127.0.0.1:5000/route/v1/driving/-56.174030195714,-34.8634731144801;-56.0461527421891,-34.7048062340561?steps=false&geometries=geojson&overview=simplified']

In [79]:
lines = []
coords = []

for url in urls:
    response = requests.get(url)
    coord = response.json()['routes'][0]['geometry']['coordinates']
    coord_tuples = [(i[1], i[0]) for i in coord]
    points = [Point(i) for i in coord]
    lines.append(LineString(points))
    coords.append(coord_tuples)

id_diadas = list(itertools.product(puntos.DEPTO, puntos.DEPTO))

In [82]:
coord_tuples

[(-33.17008, -54.34938), (-33.17008, -54.34938)]

In [83]:
df = pd.DataFrame(id_diadas, columns=['ori', 'des'])
df['coords'] = coords
df = df.loc[df.ori != df.des].reset_index(drop=True)

df.head()

,ori,des,coords
0,1,2,"[(-34.863831, -56.174277), (-34.785226, -56.27..."
1,1,3,"[(-34.863831, -56.174277), (-34.865018, -56.17..."
2,1,4,"[(-34.863831, -56.174277), (-34.827146, -56.14..."
3,1,5,"[(-34.863831, -56.174277), (-34.870376, -56.17..."
4,1,6,"[(-34.863831, -56.174277), (-34.832533, -56.25..."


In [87]:
import pickle

In [89]:
with open('data/shortest_paths.pkl', 'wb') as file:
    pickle.dump(df, file)

In [6]:
import pandas as pd
geojson_lines = pd.read_json('data/shortest_paths.geojson', orient='index')
geojson_lines

,0
type,FeatureCollection
crs,"{'type': 'name', 'properties': {'name': 'urn:o..."
features,"[{'type': 'Feature', 'properties': {'origen': ..."


In [ ]:
folium.GeoJson(antarctic_ice_edge, name="geojson").add_to(m)